In [53]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [34]:
TEST_SPLIT = 0.2

In [35]:
df = pd.read_csv("index.csv")
t = int(TEST_SPLIT*len(df))
df = df.sample(frac=1).reset_index(drop=True)
df_test = df.loc[df.index < t]
df_train = df.loc[df.index >= t]

In [ ]:
labels = list(set(list(df.label)))
labels.sort()
labels_index = {labels[i]:i for i in range(len(labels))}

In [50]:
x_train = np.stack([np.array(load_img(f)) for f in df_train.filename], axis=0)
y_train = tf.one_hot([labels_index[label] for label in df_train.label], depth=len(labels))
print(x_train.shape, y_train.shape)

x_test = np.stack([np.array(load_img(f)) for f in df_test.filename], axis=0)
y_test = tf.one_hot([labels_index[label] for label in df_test.label], depth=len(labels))
print(x_test.shape, y_test.shape)

(6264, 224, 224, 3) (6264, 32)
(1566, 224, 224, 3) (1566, 32)


In [ ]:
mobilenet = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling="max")
mobilenet.trainable = False
classifier = Dense(32, activation="softmax")
output = classifier(mobilenet.output)
model = Model(inputs=mobilenet.inputs, outputs=output)
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
model.summary()

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
 416/6264 [>.............................] - ETA: 3:17 - loss: 0.2983 - accuracy: 0.9514

<tf.Tensor: id=833116, shape=(6264, 32), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [48]:
len(labels)

32